In [2]:
import os
import sys
sys.executable

'/home/eop/code/llm/venv/bin/python'

In [3]:
from huggingface_hub import login

with open("hf_token.txt", "r") as f:
    token = f.read()

login(token=token, add_to_git_credential=True)

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

Token has not been saved to git credential helper.


Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.


In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset

# Load dataset from Hugging Face Hub
dataset_id = "Jiayi-Pan/Countdown-Tasks-3to4"
dataset = load_dataset(dataset_id, split="train")
# select a random subset of 50k samples
dataset = dataset.shuffle(seed=42).select(range(50000))

# Load tokenizer from Hugging Face Hub to format the dataset to our "r1" prompt
tokenizer = AutoTokenizer.from_pretrained(model_name)

# gemerate r1 prompt with a prefix for the model to already start with the thinking process
def generate_r1_prompt(numbers, target):
    r1_prefix = [{
        "role": "system",
        "content": "You are a helpful assistant. You first thinks about the reasoning process in the mind and then provides the user with the answer."
      },
      {
        "role": "user",
        "content": f"Using the numbers {numbers}, create an equation that equals {target}. You can use basic arithmetic operations (+, -, *, /) and each number can only be used once. Show your work in <think> </think> tags. And return the final equation and answer in <answer> </answer> tags, for example <answer> (1 + 2) / 3 </answer>." # Use <think> and <answer> tags exactly once."
      },
      {
        "role": "assistant",
        "content": "Let me solve this step by step.\n<think>"
      }]
    return {"prompt": tokenizer.apply_chat_template(r1_prefix, tokenize=False, continue_final_message=True), "target": target}

# convert our dataset to the r1 prompt
dataset = dataset.map(lambda x: generate_r1_prompt(x["nums"], x["target"]))

# split the dataset into train and test
train_test_split = dataset.train_test_split(test_size=0.1)

train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

NameError: name 'model_name' is not defined

In [ ]:
import re

def format_reward_func(completions, target, **kwargs):
    """
    Format: <think>...</think><answer>...</answer>
    Args:
        completions (list[str]): Generated outputs
        target (list[str]): Expected answers

      Returns:
          list[float]: Reward scores
    """
    rewards = []

    for completion, gt in zip(completions, target):

      try:
        # add synthetic <think> as its already part of the prompt and prefilled for the assistant to more easily match the regex
        completion = "<think>" + completion
        # Check if the format is correct
        regex = r"^<think>([^<]*(?:<(?!/?think>)[^<]*)*)<\/think>\n<answer>([\s\S]*?)<\/answer>$"

        match = re.search(regex, completion, re.DOTALL)
        print(completion)
        print("\n\n\n")
        # if the format is not correct, reward is 0
        if match is None or len(match.groups()) != 2:
            rewards.append(0.0)
        else:
            rewards.append(1.0)
      except Exception:
        rewards.append(0.0)
    return rewards

def equation_reward_func(completions, target, nums, **kwargs):
    """
    Evaluates completions based on:
    2. Mathematical correctness of the answer

    Args:
        completions (list[str]): Generated outputs
        target (list[str]): Expected answers
        nums (list[str]): Available numbers

    Returns:
        list[float]: Reward scores
    """
    rewards = []
    for completion, gt, numbers in zip(completions, target, nums):
      try:
        # add synthetic <think> as its already part of the prompt and prefilled for the assistant to more easily match the regex
        completion = "<think>" + completion
        # Check if the format is correct
        match = re.search(r"<answer>(.*?)<\/answer>", completion)
        if match is None:
            rewards.append(0.0)
            continue
        # Extract the "answer" part from the completion
        equation = match.group(1).strip()
        # Extract all numbers from the equation
        used_numbers = [int(n) for n in re.findall(r'\d+', equation)]

        # Check if all numbers are used exactly once
        if sorted(used_numbers) != sorted(numbers):
            rewards.append(0.0)
            continue
        # Define a regex pattern that only allows numbers, operators, parentheses, and whitespace
        allowed_pattern = r'^[\d+\-*/().\s]+$'
        if not re.match(allowed_pattern, equation):
           rewards.append(0.0)
           continue

        # Evaluate the equation with restricted globals and locals
        result = eval(equation, {"__builtins__": None}, {})
        print("suggested_answer: ", result, "gt: ", gt)
        # Check if the equation is correct and matches the ground truth
        if abs(float(result) - float(gt)) < 1e-5:
            rewards.append(1.0)
        else:
            rewards.append(0.0)
      except Exception:
            # If evaluation fails, reward is 0
            rewards.append(0.0)
    return rewards

In [ ]:
correct_sample_1 = """We need to find an equation using the numbers 19, 36, 55, and 7
exactly once, with basic arithmetic operations, that equals 65. One possible
combination is 55 + 36 - 19 + 7... </think>
<answer> 55 + 36 - 7 - 19 </answer>"""

correct_sample_2 = """ ... </think>
<answer> 55 + 36 - 7 - 19 </answer>"""

wrong_format = """User: Using the numbers [19, 36, 55, 7], create an equation that equals 65."""

wrong_format_2 = """To find the equation that equals 79 using the numbers 95, 78, 6, 88, I'll start by adding 88 and 95:
95 + 88 = 183
Now, let's subtract 104 from 183 to get 79:
183 - 104 = 79
<think> 183 - 104 = 79 </think><think> 183 - 104 = 79 </think><answer> 183 - 104 = 79 </answer>"""

wrong_result = """ ... </think>
<answer> 55 + 36 - 7 - 18 </answer>"""


test_rewards = format_reward_func(completions=[correct_sample_1, correct_sample_2, wrong_format, wrong_format_2, wrong_result], target=["65", "65", "65", "65", "65"], nums=[[19, 36, 55, 7]] * 5)
assert test_rewards == [1.0, 1.0, 0.0, 0.0, 1.0], "Reward function is not working"
test_rewards = equation_reward_func(completions=[correct_sample_1, correct_sample_2, wrong_format, wrong_format_2, wrong_result], target=["65", "65", "65", "65", "65"], nums=[[19, 36, 55, 7]] * 5)
assert test_rewards == [1.0, 1.0, 0.0, 0.0, 0.0], "Reward function is not working"

In [ ]:
from trl import GRPOConfig, GRPOTrainer, get_peft_config, ModelConfig

# our model we are going to use as policy
model_config = ModelConfig(
    model_name_or_path=model_name,
    torch_dtype="bfloat16",
    attn_implementation="flash_attention_2",
    use_peft=True,
    load_in_4bit=True,
)

# Hyperparameters
training_args = GRPOConfig(
    output_dir="out",
    learning_rate=5e-7,
    lr_scheduler_type="cosine",
    logging_steps=1,
    max_steps=100,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    bf16=True,
    # GRPO specific parameters
    max_prompt_length=256,
    max_completion_length=1024, # max length of the generated output for our solution
    num_generations=8,
    beta=0.001,
    report_to="wandb",
)
trainer = GRPOTrainer(
    model=model_config.model_name_or_path,
    reward_funcs=[format_reward_func, equation_reward_func],
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=get_peft_config(model_config),
)

In [ ]:
prompt = generate_r1_prompt([19, 36, 55, 7], 65)
print(prompt)
prompt_tensor = tokenizer(prompt["prompt"], return_tensors="pt")
input_ids = prompt_tensor["input_ids"].cuda()

In [ ]:
generation = trainer.model.generate(
    input_ids,
    max_length=1024,
    num_return_sequences=8,
    temperature=0.9,
    do_sample=True,
)
generation.size()

In [18]:
prompt_length = input_ids.shape[1]
generated_text = tokenizer.batch_decode(generation[:, prompt_length:], skip_special_tokens=True)
generated_text

[" Let's start by looking at our list of numbers: 19, 36, 55, 7. We need to find an equation that equals 65 using these numbers exactly once.</think>\n<think> First, let's try adding some numbers together:</think>\n<answer>(55 - 19) * (7/36) = 65</answer>",
 " I need to find two of these four numbers that when multiplied will equal 65, because division is needed. Let's try different combinations: \n- 1 * 36 = 36 (too small)\n- 1 * 55 = 55 (too big)\n- 1 * 7 = 7 (too small)\n\nSo let's try multiplying them all together:\n(19 * 36 * 55) - 7 = 65\n\nBut we have one number left, so we need to add it:\n(19 * 36 * 55 * 7) - 36 - 55 - 7 - 19 = 65\n\nTherefore, the equation that equals 65 using the numbers provided is:\n<answer>(19 * 36 * 55 * 7) - 36 - 55 - 7 - 19 = 65</answer>",
 " \nTo find an equation using the numbers [19, 36, 55, 7] to equal 65, we need to consider different combinations of these numbers and operators (+, -, *, /). We'll try to use all four numbers exactly once and ensur

In [20]:
import os
os.makedirs("out", exist_ok=True)

with open("out/generations.md", "wb") as f:
    for text in generated_text:
        f.write(("\n# Generation\n" + text + "\n\n").encode())
        format_reward = format_reward_func([text], ["65"])
        equation_reward = equation_reward_func([text], ["65"], [["19", "36", "55", "7"]])
        f.write(f"Format Reward: {format_reward}\n\n".encode())
        f.write(f"Equation Reward: {equation_reward}\n\n".encode())
        print(f"Format Reward: {format_reward}, Equation Reward: {equation_reward}")

<think> Let's start by looking at our list of numbers: 19, 36, 55, 7. We need to find an equation that equals 65 using these numbers exactly once.</think>
<think> First, let's try adding some numbers together:</think>
<answer>(55 - 19) * (7/36) = 65</answer>




Format Reward: [0.0], Equation Reward: [0.0]
<think> I need to find two of these four numbers that when multiplied will equal 65, because division is needed. Let's try different combinations: 
- 1 * 36 = 36 (too small)
- 1 * 55 = 55 (too big)
- 1 * 7 = 7 (too small)

So let's try multiplying them all together:
(19 * 36 * 55) - 7 = 65

But we have one number left, so we need to add it:
(19 * 36 * 55 * 7) - 36 - 55 - 7 - 19 = 65

Therefore, the equation that equals 65 using the numbers provided is:
<answer>(19 * 36 * 55 * 7) - 36 - 55 - 7 - 19 = 65</answer>




Format Reward: [0.0], Equation Reward: [0.0]
<think> 
To find an equation using the numbers [19, 36, 55, 7] to equal 65, we need to consider different combinations of thes

In [21]:
import wandb
run = wandb.init(
    project="LLM-RL",
    config=model_config.__dict__ | training_args.__dict__,
)
run.get_url()

In [22]:
trainer.train()

Step,Training Loss
1,-0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000


KeyboardInterrupt: 